# Dataclasses helps you  to write better code

In [2]:
import random 
import string


def generate_id():
    return "".join(random.choices(string.ascii_letters, k=12))

class Person:
    def __init__(self, name: str, address: str) -> None:
        self.id = generate_id()
        self.name = name
        self.address = address
        self.email_address = []

def main() -> None:
    person = Person(name="Cloves Paiva", address="Florianópolis-SC Brazil")
    print(person)

main()

See, when we print the class Person the output the name of the class and it address. That's is not so useful.

This class just keep persons data. Given that is a kind of class to keep data, it would be better to see the fields and the data it keeps.

We can do that by defining a data class.

In [13]:
import random 
import string
from dataclasses import dataclass, field


def generate_id():
    return "".join(random.choices(string.ascii_letters, k=12))

# kw_only=True means that the user must always include the names of
# the fields/parameters at initialization.
@dataclass(kw_only=True) # frozen=True means you can not modify the values after creation.
class Person:
    name:str
    address:str

    # the fields that has a default value must be after 
    # the ones that do not have default value.
    id: str = field(default_factory=generate_id)
    email_address: list[str] = field(default_factory=list, repr=False) # repr=false means we do not want to see the email list in print call

    # this field is a combination of address and name
    # but these field are not define yet:
    search_string: str = field(init=False)

    # but we can define how it may be set after initialization
    # like a setter
    def __post_init__(self) -> None:
        self.search_string = f"{self.name} {self.address}" 

def main() -> None:
    person = Person(name="Cloves Paiva", address="Florianópolis-SC Brazil")
    print(person)

main()

Person(name='Cloves Paiva', address='Florianópolis-SC Brazil', id='gljOtgylRurF', search_string='Cloves Paiva Florianópolis-SC Brazil')


If we need that the data class could not change value after initialization, we can set ```frozen=true```, but now we could not ```__pot_init__``` the ```search_string```, nut we can use ```property```. 

In [16]:
import random 
import string
from dataclasses import dataclass, field


def generate_id():
    return "".join(random.choices(string.ascii_letters, k=12))

# kw_only=True means that the user must always include the names of
# the fields/parameters at initialization.
@dataclass(frozen=True) # frozen=True means you can not modify the values after creation.
class Person:
    name:str
    address:str

    # the fields that has a default value must be after 
    # the ones that do not have default value.
    id: str = field(default_factory=generate_id)
    email_address: list[str] = field(default_factory=list, repr=False) # repr=false means we do not want to see the email list in print call

    # that's is also know as "getter"
    @property
    def search_string(self) -> str:
        return f"{self.name} {self.address}"

def main() -> None:
    person = Person(name="Cloves Paiva", address="Florianópolis-SC Brazil")
    print(person)
    print(person.search_string)

main()

Person(name='Cloves Paiva', address='Florianópolis-SC Brazil', id='QJEjSGuYqZwR')
Cloves Paiva Florianópolis-SC Brazil


# Setters

In [22]:
import random 
import string
from dataclasses import dataclass


def generate_id():
    return "".join(random.choices(string.ascii_letters, k=12))


@dataclass
class VideoClip:
    minutes: int
    seconds: int
    title: str

    @property
    def duration(self) -> int:
        """
        Total video duration in seconds
        """
        return self.minutes * 60 + self.seconds

    @duration.setter
    def duration(self, seconds: int) -> None:
        """
        receive total seconds and aplit it into minutes and seconds.
        """
        self.minutes, self.seconds = divmod(seconds, 60)

def main() -> None:
    video = VideoClip(minutes=10, seconds=10, title="The Video")
    print(video)
    print(video.duration)
    print(video.minutes, video.seconds)
    video.duration = 340
    print(video.duration)
    print(video.minutes, video.seconds)

main()

VideoClip(minutes=10, seconds=10, title='The Video')
610
10 10
340
5 40


# Slots

If you need to alter very often a class field, you can get better performance by setting slots true.

In [23]:
import timeit 
from dataclasses import dataclass
from functools import partial
from statistics import median

@dataclass(slots=False)
class Person:
    name: str
    address: str
    email: str

@dataclass(slots=True)
class PersonSlots:
    name: str
    address: str
    email: str

def get_set_delete(person: Person | PersonSlots):
    person.address = "Rua Delminda Silveira, 729"
    person.address
    del person.address

def main():
    person = Person("Cloves", "Av 13 de maio", "clo@gmail.com")
    person_slots = PersonSlots("Daniela", "Servidão das Palmeiras bonitas", "dani@gmail.com")
    no_slots = median(timeit.repeat(partial(get_set_delete, person), number=1_000_000))
    slots = median(timeit.repeat(partial(get_set_delete, person_slots), number=1_000_000))
    print(f"No Slots: {no_slots}")
    print(f"Slots: {slots}")
    print(f"% performance improvement: {(no_slots - slots)/ no_slots:.2%}")

main()

No Slots: 0.194428651000635
Slots: 0.12562534400058212
% performance improvement: 35.39%
